# 멀티 에이전트 네트워크를 구축하는 방법 (함수형 API)

!!! info "필수 조건" 
    이 가이드는 다음에 대한 익숙함을 가정합니다:

    - [멀티 에이전트 시스템](../../concepts/multi_agent)
    - [함수형 API](../../concepts/functional_api)
    - [Command](../../concepts/low_level/#command)
    - [LangGraph 용어집](../../concepts/low_level/)

이 how-to 가이드에서는 각 에이전트가 다른 모든 에이전트와 통신할 수 있고(다대다 연결) 다음에 호출할 에이전트를 결정할 수 있는 [멀티 에이전트 네트워크](../../concepts/multi_agent#network) 아키텍처를 구현하는 방법을 보여줍니다. [함수형 API](../../concepts/functional_api)를 사용할 것입니다 — 개별 에이전트는 작업으로 정의되고 에이전트 핸드오프는 메인 [entrypoint()][langgraph.func.entrypoint]에서 정의됩니다:

```python
from langgraph.func import entrypoint
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool


# 다른 에이전트로 핸드오프할 의도를 나타내는 도구를 정의합니다
@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """호텔 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to hotel advisor"


# 에이전트를 정의합니다
travel_advisor_tools = [transfer_to_hotel_advisor, ...]
travel_advisor = create_react_agent(model, travel_advisor_tools)


# 에이전트를 호출하는 작업을 정의합니다
@task
def call_travel_advisor(messages):
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# 멀티 에이전트 네트워크 워크플로우를 정의합니다
@entrypoint()
def workflow(messages):
    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        messages = messages + agent_messages
        call_active_agent = get_next_agent(messages)
    return messages
```

## 설정

먼저 필요한 패키지를 설치하겠습니다

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-anthropic

In [6]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

## 여행 에이전트 예제

이 예제에서는 서로 통신할 수 있는 여행 어시스턴트 에이전트 팀을 구축합니다.

2개의 에이전트를 만들 것입니다:

* `travel_advisor`: 여행 목적지 추천을 도울 수 있습니다. `hotel_advisor`에게 도움을 요청할 수 있습니다.
* `hotel_advisor`: 호텔 추천을 도울 수 있습니다. `travel_advisor`에게 도움을 요청할 수 있습니다.

이것은 완전히 연결된 네트워크입니다 - 모든 에이전트는 다른 에이전트와 통신할 수 있습니다.

먼저 에이전트가 사용할 도구 중 일부를 만들어 봅시다:

In [ ]:
import random
from typing_extensions import Literal
from langchain_core.tools import tool


@tool
def get_travel_recommendations():
    """여행 목적지에 대한 추천을 받습니다"""
    return random.choice(["aruba", "turks and caicos"])


@tool
def get_hotel_recommendations(location: Literal["aruba", "turks and caicos"]):
    """주어진 목적지에 대한 호텔 추천을 받습니다."""
    return {
        "aruba": [
            "The Ritz-Carlton, Aruba (Palm Beach)"
            "Bucuti & Tara Beach Resort (Eagle Beach)"
        ],
        "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"],
    }[location]


@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """호텔 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to hotel advisor"


@tool(return_direct=True)
def transfer_to_travel_advisor():
    """여행 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to travel advisor"

!!! note "전송 도구"

    전송 도구에서 `@tool(return_direct=True)`를 사용하고 있다는 것을 알아차렸을 것입니다. 이는 개별 에이전트(예: `travel_advisor`)가 이러한 도구가 호출되면 ReAct 루프를 조기에 종료할 수 있도록 하기 위한 것입니다. 이것이 원하는 동작입니다. 에이전트가 이 도구를 호출할 때를 감지하고 _즉시_ 다른 에이전트에게 제어를 넘기고 싶기 때문입니다.
    
    **참고**: 이것은 사전 빌드된 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]와 함께 작동하도록 되어 있습니다 -- 사용자 정의 에이전트를 구축하는 경우 `return_direct`로 표시된 도구에 대한 조기 종료 처리 로직을 수동으로 추가해야 합니다.

이제 에이전트 작업을 정의하고 이를 단일 멀티 에이전트 네트워크 워크플로우로 결합해 봅시다:

In [ ]:
from langchain_core.messages import AIMessage
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.graph import add_messages
from langgraph.func import entrypoint, task

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

# 여행 어드바이저 ReAct 에이전트를 정의합니다
travel_advisor_tools = [
    get_travel_recommendations,
    transfer_to_hotel_advisor,
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    prompt=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_travel_advisor(messages):
    # 에이전트에 대한 입력/출력 변경 등 추가 로직을 추가할 수도 있습니다.
    # 참고: 상태의 전체 메시지 기록으로 ReAct 에이전트를 호출하고 있습니다
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# 호텔 어드바이저 ReAct 에이전트를 정의합니다
hotel_advisor_tools = [get_hotel_recommendations, transfer_to_travel_advisor]
hotel_advisor = create_react_agent(
    model,
    hotel_advisor_tools,
    prompt=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_hotel_advisor(messages):
    response = hotel_advisor.invoke({"messages": messages})
    return response["messages"]


@entrypoint()
def workflow(messages):
    messages = add_messages([], messages)

    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        messages = add_messages(messages, agent_messages)
        ai_msg = next(m for m in reversed(agent_messages) if isinstance(m, AIMessage))
        if not ai_msg.tool_calls:
            break

        tool_call = ai_msg.tool_calls[-1]
        if tool_call["name"] == "transfer_to_travel_advisor":
            call_active_agent = call_travel_advisor
        elif tool_call["name"] == "transfer_to_hotel_advisor":
            call_active_agent = call_hotel_advisor
        else:
            raise ValueError(f"Expected transfer tool, got '{tool_call['name']}'")

    return messages

마지막으로 에이전트 출력을 렌더링하는 헬퍼를 정의해 봅시다:

In [ ]:
from langchain_core.messages import convert_to_messages


def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # 출력에서 부모 그래프 업데이트를 건너뜁니다
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

원래 멀티 에이전트 시스템과 동일한 입력을 사용하여 테스트해 봅시다:

In [10]:
for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "i wanna go somewhere warm in the caribbean. pick one destination and give me hotel recommendations",
        }
    ],
    subgraphs=True,
):
    pretty_print_messages(chunk)

Update from subgraph call_travel_advisor:


Update from node agent:


================================== Ai Message ==================================

[{'text': "I'll help you find a warm Caribbean destination and then get some hotel recommendations for you.\n\nLet me first get some destination recommendations for the Caribbean region.", 'type': 'text'}, {'id': 'toolu_015vT8PkPq1VXvjrDvSpWUwJ', 'input': {}, 'name': 'get_travel_recommendations', 'type': 'tool_use'}]
Tool Calls:
  get_travel_recommendations (toolu_015vT8PkPq1VXvjrDvSpWUwJ)
 Call ID: toolu_015vT8PkPq1VXvjrDvSpWUwJ
  Args:


Update from subgraph call_travel_advisor:


Update from node tools:


================================= Tool Message =================================
Name: get_travel_recommendations

turks and caicos


Update from subgraph call_travel_advisor:


Update from node agent:


================================== Ai Message ==================================

[{'text': "Based on the recommendation, I sugges

완성입니다 - `travel_advisor`가 목적지를 선택한 다음 더 많은 정보를 위해 `hotel_advisor`를 호출하기로 결정합니다!